# Working with hillslope profiles

Written by Simon M. Mudd, last update 08/11/2022

In this notebook we will use a little python package called `hillslopetoy` that was written by Simon. 
This has some functionality for importing profiles, fitting them to mesured topography, and running some scenarios of hillslope evolution. 

## Get the package

First we need to make sure the `hillslopetoy` package is installed:

In [ ]:
!pip install hillslopetoy

Now import the `hillslopetoy`, along with `matplotlib` to see the outputs.  

In [ ]:
import hillslopetoy as ht
import matplotlib.pyplot as plt
import numpy as np

## Steady state solutions

First, we will set some steady state solutions, and see what they look like. 
We can first compare some linear and nonlinear hillslope profiles, we start by setting the x locations where the divide is at `x = 0`

In [ ]:
x = ht.set_profile_locations_half_length(half_length = 20,spacing = 1)

z1 = ht.ss_nonlinear_elevation(x,C_0=0.0004,D=0.01)
z1 = ht.set_channel_to_zero(z1)

z2 = ht.ss_linear_elevation(x,C_0=0.0004,D=0.01)
z2 = ht.set_channel_to_zero(z2)

In [ ]:
plt.clf()
f, (ax1) = plt.subplots(1, 1)
ax1.plot(x, z1, label="steady nonlinear",alpha = 0.8,color="r")
ax1.plot(x, z2, label="steady linear",alpha = 0.8,color="g")

# These are just the labels for the figure
ax1.set_xlabel("Distance from divide (m)",fontsize=24)
ax1.set_ylabel("Elevation (m)",fontsize=20)

for tick in ax1.xaxis.get_major_ticks():
    tick.label1.set_fontsize(14)
for tick in ax1.yaxis.get_major_ticks():
    tick.label1.set_fontsize(14)    

plt.legend(fontsize = 14)
plt.tight_layout()

Now lets try this with arbitrary x spacing

In [ ]:
x2 = [-20,-17,-12,-11,-9.5,-7,-3,-2,-0.7,1.1,2.5,6,12,17,18.5,20]
x2 = np.asarray(x2)
print(x2)

z3 = ht.ss_nonlinear_elevation(x2,C_0=0.0004,D=0.01)
z3 = ht.set_channel_to_zero(z3)

z4 = ht.ss_linear_elevation(x2,C_0=0.0004,D=0.01)
z4 = ht.set_channel_to_zero(z4)

In [ ]:
plt.clf()
f, (ax1) = plt.subplots(1, 1)
ax1.plot(x, z1, label="steady nonlinear",alpha = 0.8,color="r")
ax1.plot(x, z2, label="steady linear",alpha = 0.8,color="g")

ax1.scatter(x2, z3, label="steady nonlinear",alpha = 0.8,color="b")
ax1.scatter(x2, z4, label="steady linear",alpha = 0.8,color="k")

# These are just the labels for the figure
ax1.set_xlabel("Distance from divide (m)",fontsize=24)
ax1.set_ylabel("Elevation (m)",fontsize=20)

for tick in ax1.xaxis.get_major_ticks():
    tick.label1.set_fontsize(14)
for tick in ax1.yaxis.get_major_ticks():
    tick.label1.set_fontsize(14)    

plt.legend(fontsize = 14)
plt.tight_layout()

## Getting a profile from QGIS

In one of the class practials we showed how to extract a profile using QGIS.
We are going to use this to extract a profile for the `hillslopetoy`. 

Here are the steps

1. Open the topographic dataset (for the class this is `el_study.bil`)
2. Make a contour map (Raster -> Extraction -> Contour). he purpose of the contour map is to help you choose a transect. The `hillslopetoy` assumes that you have a 1D hillslope*. This means that you should select transects where the contour lines do not spread apart too much.
3. Use the profile tool to make a transect from the hilltop to the channel. You want to draw a transect that is perpendicular to the contour lines in a place where the contour lines are roughly parallel to each other. **Make sure you start the transect at the divide and not the river** (since the `hillslopetoy` assumes the divide is at `x=0`).
4. Then click on table, and copy to clipboard with coordinates. 
5. Paste this into excel. 
6. Add the headers: x, easting, northing, elevation
7. Save to csv. Call it something sensible. 
8. I have added an example file for the use with this notebook. 



*(the one dimension is distance, or `x`, the elevation is a dependant variable, that is `z` depends on `x`, so it isn't counted as a second dimension). 

## Load the transect

Okay, lets load the transect. This one is included in this teaching notebook but you can extract your own transect using the method described above. We are going to use `pandas` to look at the data so we have to import that before we get the data.  

In [ ]:
import pandas as pd

In [ ]:
# Read some csv data into a pandas dataframe. 
transect_df = pd.read_csv("el_transect.csv")
transect_df.head()

Lets have a look at what the data look like:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(transect_df.x,transect_df.elevation)
plt.tight_layout()

## Fitting the transect the stupid way

Okay, we are going to try to constrain the parameters that create a steady state landscape here. 

There are a few ways to do this, but I am going to use a fancy way of using something called the `optimize` functions in python, that are used for fitting models. In this case you have data (the transect), the "model" (the `hillslopetoy`) and the parameters you want to fit (the sediment transport coefficient, `D`, and a second parameter that controls the absolute elevation of the transect, `c`). 

Lets try it with the whole transect first:

In [ ]:
Erosion_rate = 0.0001

# We need to guess the parameters. For c you can use the elevation of the channel
D_guess = 0.01
c_guess = 260

x = transect_df.x
y = transect_df.elevation

In [ ]:
from scipy.optimize import least_squares

def model(u,x,C_0):
    z = ht.ss_nonlinear_elevation(x,C_0=C_0,D=u[0],c=u[1])
    return z

def fun(u,x,C_0,y):
    return model(u,x,C_0) - y

u0 = np.array([D_guess,c_guess])
res = least_squares(fun, u0, args=(x, Erosion_rate, y), verbose=1)

x_test = np.linspace(0, 450)
y_test = model(res.x, x_test, C_0)

plt.plot(x, y, 'o', markersize=4, label='data')
plt.plot(x_test, y_test, label='fitted model')
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc='lower right')
plt.show()

## A more clever fit

Wait a minute. The bottom of the transect is super steep and we think this might indicate transience. So we probably only want to fit the upper part of the transect. Lets remove the tow of this slope. The steep bit starts at around x = 380 m so we can use pandas to remove anything greater than that. 

In [ ]:
short_transect_df = transect_df.drop(transect_df[transect_df.x>380].index)

print(short_transect_df.x)

In [ ]:
# We need to guess the parameters. For c you can use the elevation of the channel
D_guess = 0.01
c_guess = 260

x = short_transect_df.x
y = short_transect_df.elevation

u0 = np.array([D_guess,c_guess])
res = least_squares(fun, u0, args=(x, Erosion_rate, y), verbose=1)

x_test = np.linspace(0, 380)
y_test = model(res.x, x_test, C_0)

plt.plot(x, y, 'o', markersize=4, label='data')
plt.plot(x_test, y_test, label='fitted model')
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc='lower right')
plt.show()

In [ ]:
y

In [ ]:
model(x, u)

In [ ]:
x

In [ ]:
res = least_squares(fun, x0, bounds=(0, 100), args=(u, y), verbose=1)